In [1]:
import pyarrow as pa
from collections import defaultdict
import math
import pandas as pd
import pyarrow.parquet as pq
import random
import itertools
import numpy as np
import random
from tqdm import tqdm_notebook,tqdm 
from sklearn import tree

In [52]:
# Where the downloaded data are
input_path = './'
# Where to store results
output_path = './'

# Read a single day to train model on as Pandas dataframe
data0 = pq.read_table('date=2018-02-01').to_pandas()
data0.head(10)
data = data0

In [31]:
data1 = pq.read_table('date=2018-02-02').to_pandas()
print(len(data1))


363326


In [20]:
#data1.head(10)
#data1.to_csv('data=2018-02-01')

In [32]:
frames = [data0, data1]

In [44]:
data = pd.concat(frames).reset_index(drop = True)
print(len(data))
data.head(5)

723456


,instanceId_userId,instanceId_objectType,instanceId_objectId,audit_pos,audit_clientType,audit_timestamp,audit_timePassed,audit_experiment,audit_resourceType,metadata_ownerId,...,auditweights_userOwner_USER_INTERNAL_UNLIKE,auditweights_userOwner_USER_PRESENT_SEND,auditweights_userOwner_USER_PROFILE_VIEW,auditweights_userOwner_USER_SEND_MESSAGE,auditweights_userOwner_USER_STATUS_COMMENT_CREATE,auditweights_userOwner_VIDEO,auditweights_userOwner_VOTE_POLL,auditweights_x_ActorsRelations,auditweights_likersSvd_spark_hyper,auditweights_source_PROMO
0,138,Post,16788021,1,API,1517512273619,3651351,XPRM-5386_q2x3,8,2154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,222,Post,23852723,33,WEB,1517495744978,3937907,XPRM-5386_G2,8,23080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,384,Post,12454221,15,WEB,1517508395567,1517508395567,XPRM-5386_q2x2,8,39056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,666,Post,18335103,13,WEB,1517486842005,1517486842005,XPRM-5386_G0,8,69277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1209,Post,22920031,2,MOB,1517498414790,2031754,XPRM-5386_G1,8,23941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1024.0,NaN,NaN


In [34]:
data = data.reset_index(drop = True)
data.head(5)

,instanceId_userId,instanceId_objectType,instanceId_objectId,audit_pos,audit_clientType,audit_timestamp,audit_timePassed,audit_experiment,audit_resourceType,metadata_ownerId,...,auditweights_userOwner_USER_INTERNAL_UNLIKE,auditweights_userOwner_USER_PRESENT_SEND,auditweights_userOwner_USER_PROFILE_VIEW,auditweights_userOwner_USER_SEND_MESSAGE,auditweights_userOwner_USER_STATUS_COMMENT_CREATE,auditweights_userOwner_VIDEO,auditweights_userOwner_VOTE_POLL,auditweights_x_ActorsRelations,auditweights_likersSvd_spark_hyper,auditweights_source_PROMO
0,138,Post,16788021,1,API,1517512273619,3651351,XPRM-5386_q2x3,8,2154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,222,Post,23852723,33,WEB,1517495744978,3937907,XPRM-5386_G2,8,23080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,384,Post,12454221,15,WEB,1517508395567,1517508395567,XPRM-5386_q2x2,8,39056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,666,Post,18335103,13,WEB,1517486842005,1517486842005,XPRM-5386_G0,8,69277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1209,Post,22920031,2,MOB,1517498414790,2031754,XPRM-5386_G1,8,23941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1024.0,NaN,NaN


In [128]:

f0 = ['instanceId_userId','auditweights_svd_prelaunch', 
        'auditweights_ctr_high', 
        'auditweights_ctr_gender', 
        'auditweights_friendLikes',
         'auditweights_numLikes','auditweights_numShows','auditweights_numDislikes']
f1 = [ 'auditweights_userAge', 'auditweights_ageMs', 'auditweights_ctr_negative', 'auditweights_dailyRecency', 
      'userOwnerCounters_CREATE_LIKE', 'userOwnerCounters_TEXT', 'userOwnerCounters_IMAGE']
f2 = ['audit_pos' ,'user_gender','user_status']
fy = ['feedback']


In [ ]:
#data['userOwnerCounters_PHOTO_COMMENT_CREATE']
#data.iloc[100:110,140:]

# Построение векторов признаков:

1. Выбираются только пользователи, имеющие разные виды взаимодействие (0, 1) и количество этих взаимодействий больше 1. 

2. Для каждого пользователя создаются всевозможные пары, вычисляются разности этих векторов (без учета того, чтобы вектора были с разными feedback).

3. Следом проеобразование признаков в бинарный формат: признак > 0 => 1, : признак <= 0 => 0

4. Для feedback есть два варианта: 

     1) {1,0,-1} 
 
     2) {0,1} - просто исходя из того, что левый вектор должен быть больше чем правый



In [25]:

def entropy(data):
    labels = defaultdict(int)
    N = len(data)
    # подсчет меток
    for i in range(N):
        line = data.loc[i]
        if (line[-1] not in labels.keys()):
            labels[line[-1]] = 0
        labels[line[-1]]+=1
        
    entropy = 0
    #print(labels)
    for key in labels:
        p = labels[key]/N
        entropy -=p * math.log(p,2)
        
    return (entropy)
    
def choose_feature(data):
    H = entropy(data)
    #print(H)
    N = len(data)
    features = list(data.columns.values)[:-1]
    best_feature = features[0]
    max_gain = 0
    for feature in features:
        names = data.loc[:,feature]
        names = set(names) #uniq atributes of feature {0,1}
        entr = 0
        for name in names:
            new_X = data.loc[data[feature]==name].reset_index(drop = True) #name = {0,1} 
            p = len(new_X) / N
            entr+= p * entropy(new_X)
        gain = H - entr
        if (gain > max_gain):
            max_gain = gain
            best_feature = feature   
    return(best_feature)
        
       
    
def get_random_pairs(numbers):
      # Generate all possible non-repeating pairs
        pairs = list(itertools.combinations(numbers, 2))
 
      # Randomly shuffle these pairs
        random.shuffle(pairs)
        return (pairs)           
            
        
            
def prevaling_mark(marks):
    uniq_marks = list(set(marks))
    max_count = 0
    best_mark = marks[0]
    for mark in uniq_marks:
        current = marks.count(mark) 
        if(current> max_count):
            max_count = current
            best_mark = mark
            
    return(best_mark)   
    
def Dtree(data):
    H = entropy(data)
    marks = [data.loc[i][-1] for i in range(len(data))]
    #print(data)
    features = list(data.columns.values)[:-1]
    #marks should be different
    if (len(set(marks)) ==1 ):
        return(marks[0])
    
    #check last feature
    if (len(features)==0):
        return(prevaling_mark(marks))
    #best_feature = string name of feature
    best_feature = choose_feature(data)
    #print(best_feature)
    #build a node with best feature
    
    tree = {best_feature:{}}
    
    uniq_feature_names = list(set(data[best_feature]))
    #print("uniq_feature_names ", uniq_feature_names )
    for name in uniq_feature_names:
        new_data = data[data[best_feature]==name].reset_index(drop = True)
        new_data = new_data.drop([best_feature], axis = 1)
        #print(name)
        tree[best_feature][name] = Dtree(new_data)
        #print(tree)
        
    return(tree)
    

    
def predict_target(tree, row):
    #print(type(tree))
    if (type(tree) == dict):
        feature = list(tree.keys())[0]
        #print(feature)
        value = row[feature]
        #print(value)
        new_tree = tree[feature][value]
        new_tree = predict_target(new_tree, row)
        return(new_tree)
       
    else :
        return (tree)   
    
    
def print_tree(tree, level):
    if (type(tree) == dict):
        feature = list(tree.keys())[0]
        print("\t"*level+str(feature) + ":")
        level+=1
        values = list(tree[feature].keys())
        for i in values:
            print("\t"*level +str(i))
            print_tree(tree[feature][i],level)
    else:
        print("\t"*level, tree)
        
    

In [148]:
def processing(data,feature_names):

    X = pd.DataFrame(columns = [feature_names]) #empty
    
    user_ids = list(data["instanceId_userId"])
    user_ids = list(set(user_ids))
    #print(len(user_ids))
    for user in tqdm_notebook(user_ids):
        #all posts of user
        posts = data.loc[data["instanceId_userId"]==user].reset_index(drop = True)
        #drop column with user id
        posts = posts.drop(["instanceId_userId"], axis = 1)
        
        #проверка на длину, на разные ответные реакции
        N = len(posts)
        #print(user, N)
        Nnegative = len(posts.loc[posts["feedback"] == 0])
        Npositive = len(posts.loc[posts["feedback"] == 1])
        #print(posts)
        #проверка 
        if ( N>1)&(Nnegative!=N)&(Npositive!=N):
                s = list(range(N)) # номера строк постов конкреного юзера
                random.shuffle(s) # << shuffle before print or assignment
                #получение всевозможных комбинаций
                pairs = get_random_pairs(s) #pairs = [(1,2), (1,0) ....]

                for i in pairs:
                    new_row = posts.loc[i[0]] - posts.loc[i[1]]
                    X = X.append(new_row, ignore_index=True)
    return (X)


In [149]:
def make_binary3(X):
        #преобразование в бинарную матрицу признаков, feedback = {-1,0,1}
        X_feature = (X[feature_names[:-1]]>0)*1

        y = X['feedback'] # feedback = {-1,0,1}
        X = pd.concat([X_feature, y],axis = 1)
        return(X)
    
def make_binary2(X):
    #преобразование в бинарную матрицу признаков, feedback = {-0,1}
    return((X[feature_names]>0)*1)


#Train / test splirring
def split(X, test_size):
    indexes = [i for i in range(len(X))]

    random.shuffle(indexes)
    test_indexes = indexes[:test_size]
    train_indexes = indexes[test_size + 1:]
    Xtrain = X.loc[train_indexes].reset_index(drop = True)
    Xtest = X.loc[test_indexes]
    #split x and y
    Ytest = Xtest['feedback'].reset_index(drop = True)
    Xtest = Xtest[feature_names[:-1]].reset_index(drop = True)
    
    return(Xtrain, Xtest, Ytest)


In [167]:
feature_names = []
feature_names.extend(f0)
feature_names.extend(f1)
#feature_names.extend(f2)
feature_names.extend(fy)
print("done")

X = data[feature_names[:-1]].fillna(0.0)
print("fillna done")
y = data['feedback'].apply(lambda x: 1.0 if("Liked" in x) else 0.0)
print("y done")
new_data  = pd.concat([X,y], axis = 1)
print(" new_data done")
#feature_names.remove('instanceId_userId')

X = processing(new_data, feature_names.remove('instanceId_userId'))
X.head(5)
print("processing done")
X = make_binary2(X)
X.head(5)
print("make_binary2 done")
#test train split
Xtrain, Xtest, Ytest = split(X, 1000)
print("split done")

#Tree = Dtree(Xtrain)


done
fillna done
y done
 new_data done


processing done
make_binary2 done
split done


In [168]:
len(Xtrain)

51963

In [169]:
Tree = Dtree(Xtrain)
#print_tree(Tree,0)


In [170]:
Xtrain.head(5)

,auditweights_svd_prelaunch,auditweights_ctr_high,auditweights_ctr_gender,auditweights_friendLikes,auditweights_numLikes,auditweights_numShows,auditweights_numDislikes,auditweights_userAge,auditweights_ageMs,auditweights_ctr_negative,auditweights_dailyRecency,userOwnerCounters_CREATE_LIKE,userOwnerCounters_TEXT,userOwnerCounters_IMAGE,feedback
0,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0
1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0
2,0,1,0,0,1,1,1,0,1,1,0,0,1,1,1
3,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0
4,1,1,1,0,0,0,0,0,1,1,0,1,0,0,1


In [171]:
len(Xtest)

1000

# prediction

In [172]:
y_pred= []
for i in range(len(Xtest)):
    row = Xtest.loc[i]
    y_pred.append(predict_target(Tree, row))

In [173]:
#accuracy

sum(y_pred == Ytest) / len(y_pred)

0.746

 ## Decision tree from sklearn 

In [174]:
from sklearn import tree
x = Xtrain[feature_names[:-1]]
y = Xtrain['feedback']

clf = tree.DecisionTreeClassifier()
clf = clf.fit(x, y)
y_pred_sklearn = clf.predict(Xtest)

In [175]:
sum((Ytest) == y_pred_sklearn)/len(y_pred_sklearn)

0.746

In [122]:
d = data[feature_names[:-1]].fillna(0.0)[:10]
d

,auditweights_svd_prelaunch,auditweights_ctr_high,auditweights_ctr_gender,auditweights_friendLikes,auditweights_numLikes,auditweights_numShows,auditweights_numDislikes,auditweights_userAge,auditweights_ageMs,auditweights_ctr_negative,auditweights_dailyRecency,userOwnerCounters_CREATE_LIKE,userOwnerCounters_TEXT,userOwnerCounters_IMAGE
0,0.603160,0.065857,0.009836,0.0,8.0,815.0,1.0,44.0,35742344.0,0.001093,0.973046,0.826869,0.000000,0.000000
1,0.675362,0.135539,0.023984,0.0,6064.0,223594.0,1036.0,70.0,130239136.0,0.004631,0.923622,0.545485,0.545485,0.000000
2,0.761416,0.060081,0.009776,0.0,46.0,4094.0,6.0,110.0,83072064.0,0.001431,0.954284,2.402731,0.960400,0.000000
3,0.851362,0.038051,0.002660,0.0,0.0,652.0,3.0,62.0,5533506.0,0.003989,0.980847,0.000000,0.000000,0.000000
4,0.887637,0.194643,0.005013,0.0,2784.0,71321.0,325.0,46.0,1892124.0,0.004550,0.981623,0.000000,0.000000,0.000000
5,0.702343,0.172163,0.016191,1.0,47.0,1197.0,5.0,62.0,31369236.0,0.003855,0.974343,0.000000,0.000000,0.000000
6,0.690506,0.146267,0.005740,1.0,23474.0,802310.0,2862.0,37.0,55604936.0,0.003567,0.966315,0.000000,0.800732,0.000000
7,0.761575,0.127646,0.022038,1.0,536.0,20909.0,119.0,57.0,1865190.0,0.005664,0.981629,0.941192,0.000000,0.000000
8,0.820620,0.086106,0.013726,1.0,154.0,9007.0,36.0,57.0,1975772.0,0.003953,0.981605,7.351515,0.000000,0.000000
9,0.245339,0.054828,0.004651,0.0,0.0,330.0,0.0,35.0,238282.0,0.000000,0.981965,2.393331,1.725282,10.422718


In [81]:
len(d.loc[0])

14

In [123]:

len(a.columns)
a

,auditweights_svd_prelaunch,auditweights_ctr_high,auditweights_ctr_gender,auditweights_friendLikes,auditweights_numLikes,auditweights_numShows,auditweights_numDislikes,auditweights_userAge,auditweights_ageMs,auditweights_ctr_negative,auditweights_dailyRecency,userOwnerCounters_CREATE_LIKE,userOwnerCounters_TEXT,userOwnerCounters_IMAGE,feedback


In [136]:
feature_names = []
feature_names.extend(f0)
feature_names.extend(f1)
#feature_names.extend(f2)
feature_names.extend(fy)
a = pd.DataFrame(columns = [feature_names.remove('instanceId_userId')])
print(d.loc[0])
print("------")
print(a.columns)
a.append(d.loc[0],ignore_index = True)

auditweights_svd_prelaunch       6.031605e-01
auditweights_ctr_high            6.585748e-02
auditweights_ctr_gender          9.836066e-03
auditweights_friendLikes         0.000000e+00
auditweights_numLikes            8.000000e+00
auditweights_numShows            8.150000e+02
auditweights_numDislikes         1.000000e+00
auditweights_userAge             4.400000e+01
auditweights_ageMs               3.574234e+07
auditweights_ctr_negative        1.092896e-03
auditweights_dailyRecency        9.730464e-01
userOwnerCounters_CREATE_LIKE    8.268691e-01
userOwnerCounters_TEXT           0.000000e+00
userOwnerCounters_IMAGE          0.000000e+00
Name: 0, dtype: float64
------
Index([None], dtype='object')


,None,auditweights_ageMs,auditweights_ctr_gender,auditweights_ctr_high,auditweights_ctr_negative,auditweights_dailyRecency,auditweights_friendLikes,auditweights_numDislikes,auditweights_numLikes,auditweights_numShows,auditweights_svd_prelaunch,auditweights_userAge,userOwnerCounters_CREATE_LIKE,userOwnerCounters_IMAGE,userOwnerCounters_TEXT
0,NaN,35742344.0,0.009836,0.065857,0.001093,0.973046,0.0,1.0,8.0,815.0,0.60316,44.0,0.826869,0.0,0.0


In [131]:
feature_names

['auditweights_svd_prelaunch',
 'auditweights_ctr_high',
 'auditweights_ctr_gender',
 'auditweights_friendLikes',
 'auditweights_numLikes',
 'auditweights_numShows',
 'auditweights_numDislikes',
 'auditweights_userAge',
 'auditweights_ageMs',
 'auditweights_ctr_negative',
 'auditweights_dailyRecency',
 'userOwnerCounters_CREATE_LIKE',
 'userOwnerCounters_TEXT',
 'userOwnerCounters_IMAGE',
 'feedback']